Model mean sea level - nowcasts

* What is the mean sea level of the model?
* Is it close to 0? 
* Trying to determine if z=0 in the model corresponds to mean sea level. 

In [1]:
import numpy as np
import netCDF4 as nc

from salishsea_tools import tidetools
from salishsea_tools.nowcast import analyze, figures
import datetime

In [2]:
b = '/data/nsoontie/MEOPAR/NEMO-forcing/grid/bathy_meter_SalishSea.nc'

grid_B = nc.Dataset(b)
X=grid_B.variables['nav_lon'][:]
Y=grid_B.variables['nav_lat'][:]
bathy =grid_B.variables['Bathymetry'][:]

to=datetime.datetime(2014,11,1)
tf=datetime.datetime(2015,10,1)
files = analyze.get_filenames(to,tf,'1d','grid_T','/data/dlatorne/MEOPAR/SalishSea/nowcast/')

Look up mean sea level at a few points in the model.

In [3]:
names = ['Point Atkinson','Campbell River',  'Victoria', 'Neah Bay', 'Cherry Point', 'Friday Harbor',
        'Sandheads']

print('Date range: {} to {} '.format(to.strftime('%d-%b-%Y'), tf.strftime('%d-%b-%Y')))
for name in names:
    lat = figures.SITES[name]['lat']
    lon = figures.SITES[name]['lon']

    j, i = tidetools.find_closest_model_point(lon,lat,X,Y,bathy) 

    ssh, time =analyze.combine_files(files,'sossheig','None',j,i)
    
    print('{} mean ssh: {}'.format(name, np.mean(ssh)))

Date range: 01-Nov-2014 to 01-Oct-2015 
Point Atkinson mean ssh: 0.1289057433605194
Campbell River mean ssh: -0.00058242172235623
Victoria mean ssh: 0.011352158151566982
Neah Bay mean ssh: -0.0375010147690773
Cherry Point mean ssh: 0.07324444502592087
Friday Harbor mean ssh: 0.023417768999934196
Sandheads mean ssh: 0.11173241585493088


Do these values reflect anamolies unqiue to this time period? Probably not, because I would expect them to be closer together. 

How do the actual water levels compare with a long term mean sea level?

In [4]:
DFO = ['Point Atkinson', 'Campbell River', 'Victoria']

print('Date range: {}  to {} '.format(to.strftime('%d-%b-%Y'), tf.strftime('%d-%b-%Y')))

for name in DFO:
    obs = figures.load_archived_observations(name,to.strftime('%d-%b-%Y'), tf.strftime('%d-%b-%Y'))
    print('{} mean sea level - long term mean: {}'.format(name, np.mean(obs.wlev) - figures.SITES[name]['msl']))

NOAA = ['Neah Bay', 'Cherry Point', 'Friday Harbor']    
for name in NOAA:
    obs = figures.get_NOAA_wlevels(figures.SITES[name]['stn_no'],to.strftime('%d-%b-%Y'), tf.strftime('%d-%b-%Y'),
                                  product='hourly_height')
    print('{} mean sea level of observations {}'.format(name, np.mean(obs.wlev) ))

Date range: 01-Nov-2014  to 01-Oct-2015 
Point Atkinson mean sea level - long term mean: 0.07400998128508052
Campbell River mean sea level - long term mean: -0.04736726546906844
Victoria mean sea level - long term mean: 0.0838099148418523
Neah Bay mean sea level of observations -0.015930638722554895
Cherry Point mean sea level of observations 0.03153443113772447
Friday Harbor mean sea level of observations 0.06687375249500965


* I think I will have to keep an eye on this. The longer time series I have, the less I should have to compare with a long term msl in the observations.
* spinups are useless because we used the atmospheric pressure incorrectly. 
